### Сравнение библиотек для извлечение NER 

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import pymorphy2

import glob
import re
import natasha
from natasha.markup import show_markup, show_json
from polyglot.text import Text
import spacy
import polyglot
import nltk


import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data = []
for df in glob.glob('/proj_news_viz/nlp/topic_models/data/*.csv'):
    data = pd.read_csv(df, encoding='utf-8')

data = data.drop_duplicates('url')

Для сравнения из 10 текстов были извлечены имена и фамилии. 

In [12]:
texts = '\\n'.join(data.text[:10])

correct_names = ['Александр Усик', 'Тони Белью', 'Тони Белью', 'Дэвида Хэя', 'Гассиев',
           'Усик', 'Мурата Гассиева','Эммануэль Нахшон','Эдена Азара', 
           'Маурицио Сарри', 'Азар', 'Сарри', 'Азара', 'Азар' ,'Андрей Рябинский',
           'Петра Порошенко', 'Муратом Гассиевым' , 'Александром Усиком',
           'Порошенко', 'Саша', 'Усик', 'Данияра Алимбаева', 'Данияр', 'Алимбаев', 
           'Денис Тен', 'Тен', 'Доминик Рааб', 'Дэвид Дэвис', 'Терезы Мэй',
           'Борис Джонсон', 'Мэй', 'Тереза Мэй', 'Петр Порошенко', 'Александра Усика',
           'Мурата Гассиева', 'Саша', 'Усик'] 

len(correct_names)

37

In [13]:
stopword_ru=nltk.corpus.stopwords.words('russian')
morph = pymorphy2.MorphAnalyzer()

def polyglot_extract_names(texts):
    blob = polyglot.text.Text(texts,hint_language_code='ru')
    persons = []
    for entity in blob.entities:
        if entity.tag =='I-PER':
            persons.append(' '.join(entity))
    return persons


def natasha_extract_names(texts):
    extractor = natasha.NamesExtractor()
    matches = extractor(texts)
    persons = []
    spans = []
    spans.extend(_.span for _ in matches)
    for i,j in spans:
        persons.append(texts[i:j])
    return persons

def spacy_extract_names(texts):
    nlp = spacy.load('xx')
    doc = nlp(texts)
    persons = []
    for ent in doc.ents:
        if ent.label_ == 'PER':
            persons.append(ent.text)
    return persons

def evalute_NER(correct_names, pred_names):
    cnt_correct_names = 0
    error_rate = 0
    for name in correct_names:
        if name in pred_names:
            cnt_correct_names += name in pred_names
    error_rate = (len(pred_names) - cnt_correct_names)/len(pred_names)
    return (cnt_correct_names/len(correct_names), error_rate)

def pymorphy_lemmatize_text(text):
    words = []
    for word in clean_text(text):
        words.append(morph.parse(word)[0].normal_form)
    return ' '.join(words)

def clean_text(text): 

    #убираем мусор
    text = nltk.re.sub('-\s\r\n\s+|-\s\r\n|\r\n|[«»]|[><]', ' ', text.lower())    
    text = nltk.re.sub('[0-9]|[.,:;_%©?*!@#№$^•·&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-|–|—|', ' ', text)
    text = re.sub(r"\d+", ' ', text, flags=re.UNICODE)
    text = re.sub('\’|\”|‘', ' ', text)
    text = re.sub(r'\\n', '', text)
    text = re.sub(r'ё', 'е', text)
    text = re.sub('[A-z]','', text)

    #убираем стоп-слова
    words = text.split()
    words = [w for w in words if not w in stopword_ru]

    #джойним слова
    return words


Для оценки использовалась метрика accuracy и error rate ( (кол-во найденных имен  - кол-во правильных имен)/кол-во найденных имен) )

In [14]:
print('Correct names: \n', ', '.join(correct_names))

Correct names: 
 Александр Усик, Тони Белью, Тони Белью, Дэвида Хэя, Гассиев, Усик, Мурата Гассиева, Эммануэль Нахшон, Эдена Азара, Маурицио Сарри, Азар, Сарри, Азара, Азар, Андрей Рябинский, Петра Порошенко, Муратом Гассиевым, Александром Усиком, Порошенко, Саша, Усик, Данияра Алимбаева, Данияр, Алимбаев, Денис Тен, Тен, Доминик Рааб, Дэвид Дэвис, Терезы Мэй, Борис Джонсон, Мэй, Тереза Мэй, Петр Порошенко, Александра Усика, Мурата Гассиева, Саша, Усик


In [7]:
start = np.datetime64('now')
functions = [polyglot_extract_names, natasha_extract_names, 
             spacy_extract_names]
for fun in functions:
    result = fun(texts)
    metrics = evalute_NER(correct_names, result)
    print(fun.__name__)
    print(f'Accuracy: {metrics[0]}\nError rate: {metrics[1]}')
    print('Found names: \n', ', '.join(result))
    print(np.datetime64('now') - start)
          

polyglot_extract_names
Accuracy: 0.8648648648648649
Error rate: 0.30434782608695654
Found names: 
 Александр Усик, Тони Белью, Тони Белью, Дэвида Хэя, Гассиев, Мурата Гассиева, Украинец, Эммануэль Нахшон, Белых касок, Эдена Азара, Mirror ., лондонцев Маурицио Сарри, Сарри, Азар, Андрей Рябинский, Петра Порошенко, Муратом, Александром Усиком, Порошенко, Саша, Усик, Гарт Брукс, Бруно Марс, Эд Ширан, Леди Гага, Билли Джоэл, Роджер Уотерс, Coldplay, The, Кендрик Ламар, Пол Маккартни, Джордж Клуни, Алимбаев, Денис Тен, Тен, Доминик Рааб, Дэвид Дэвис, Терезы Мэй, Борис Джонсон, Мэй, Тереза Мэй, Беспилотники, Петр Порошенко, Александра Усика, Мурата Гассиева, Саша
29 seconds
natasha_extract_names
Accuracy: 0.7027027027027027
Error rate: 0.46938775510204084
Found names: 
 Александр Усик, Тони, Белью, Тони, Белью, Дэвида, Мурата Гассиева, Белых, Эммануэль, Белых, Эдена Азара, Маурицио Сарри, Сарри, Азара, Азар, Андрей Рябинский, Петра Порошенко, Муратом Гассиевым, Александром Усиком, Порошенко,

In [9]:
df_names = pd.DataFrame(columns=['full_name', 'frequency'])
df_names = df_names.set_index('full_name')

In [23]:
start = np.datetime64('now')
for i, text in enumerate(data.text[:100]):

    tmp_names = pd.DataFrame(columns=['full_name', 'frequency'])
    names = polyglot_extract_names(text)
    
    for name in names:
        lem_text = pymorphy_lemmatize_text(name)
        if (lem_text in tmp_names.full_name.values):
            ind = tmp_names.full_name == lem_text
            tmp_names.loc[ind,'frequency'] = tmp_names[ind].frequency + 1
       
        elif sum(tmp_names.full_name.apply(lambda x: lem_text.split(' ')[-1]in x.split(' '))):
            
            ind = tmp_names.full_name.apply(lambda x: lem_text.split(' ')[-1]in x.split(' '))
            tmp_names.loc[ind, 'frequency'] =  tmp_names[ind].frequency + 1
            
        else:
            tmp_names = tmp_names.append({'full_name': lem_text, 
                                        'frequency': 1}, ignore_index=True)
    tmp_names = tmp_names.set_index('full_name')
    
    df_names = pd.DataFrame(pd.concat([df_names,
                                       tmp_names]).groupby(level=0)['frequency'].sum())
            
np.datetime64('now') - start

numpy.timedelta64(7,'s')

In [18]:
df_names = df_names.reset_index()
df_names.columns = ['full_name', 'frequency']
df_names = df_names.drop(df_names[df_names.full_name.apply(lambda x: len(str(x))<4)].index)
df_names['full_name'] = df_names.full_name.apply(lambda x: ' '.join(x.split(' ')[-2:]) if len(x.split(' '))>2 else ''.join(x.strip()))

In [19]:
df_names = pd.DataFrame(df_names.groupby('full_name')['frequency'].sum()).sort_values('frequency', ascending=False)

In [20]:
df_names[df_names.frequency>5].sort_values('frequency', ascending=False)

,frequency
full_name,
александр усик,22
дональд трамп,18
андрей воробьев,16
владимир путин,14
мурат гассий,12
абель эрнандес,10
владимир познер,10
мбапп,10
карли кларк,10


In [ ]:
df_names.to_csv('data/names.csv')